In [18]:
import torch
import os

from train import train
import priors
import utils

import numpy as np

from datasets import load_openml_list, valid_dids_classification, test_dids_classification
from tabular import evaluate, get_model, get_default_spec

from tabular import bayes_net_metric, gp_metric, knn_metric, ridge_metric, catboost_metric, xgb_metric, logistic_metric, tabnet_metric

In [19]:
%load_ext autoreload

%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Loading Datasets

In [8]:
### Loads small list of datasets
print('Loading test datasets...')
test_datasets, test_datasets_df = load_openml_list(test_dids_classification[0:2], filter_for_nan=True)
ds = test_datasets

print('\n Loading valid datasets...')
valid_datasets, valid_datasets_df = load_openml_list(valid_dids_classification[0:2], filter_for_nan=True)

Loading test datasets...
wine 973
covertype 1596

 Loading valid datasets...
ionosphere 59


In [7]:
### Loads all datasets
print('Loading test datasets...')
test_datasets, test_datasets_df = load_openml_list(test_dids_classification, filter_for_nan=True)
ds = test_datasets

print('\n Loading valid datasets...')
valid_datasets, valid_datasets_df = load_openml_list(valid_dids_classification, filter_for_nan=True)

Loading test datasets...
kr-vs-kp 3
credit-g 31
vehicle 54
wine 973
kc1 1067
airlines 1169
bank-marketing 1461
blood-transfusion-service-center 1464
phoneme 1489
covertype 1596
numerai28.6 23517
connect-4 40668
car 40975
Australian 40981
segment 40984
jungle_chess_2pcs_raw_endgame_complete 41027
sylvine 41146
MiniBooNE 41150
dionis 41167
jannis 41168
helena 41169

 Loading valid datasets...
haberman 43
ionosphere 59
sa-heart 1498
cleve 40710


## Setting params

In [3]:
device = 'cpu'

In [4]:
# After how many training samples should evaluatuion be done?
# Trained models have not been trained to evaluate after 30 samples
# so performance will drop
eval_positions = [30]

# What is the maximum number of features?
# Pretrained models have to use 60
max_features = 60

# How many samples should be loaded for one dataset?
# Samples after the training sequence are used for evaluation
seq_len = 100

# How many subsamples of datasets should be drawn for each dataset
max_samples = 20

In [5]:
gp_model_checkpoint_dir = "results/tabular_model_gp.ckpt"
gp_model_config = {'batch_size': 512,
 'bptt': 100,
 'dropout': 0.5,
 'emsize': 512,
 'epochs': 100,
 'eval_positions': [10, 20, 40, 80],
 'lr': 6.271726842985807e-05,
 'nhead': 4,
 'nhid_factor': 2,
 'nlayers': 5,
 'num_features': 60,
 'prior_lengthscale': 0.00014803074521613278,
 'prior_noise': 0.001,
 'prior_normalize_by_used_features': True,
 'prior_num_features_used_sampler': {'uniform_int_sampler_f(1,max_features)': '<function <lambda>.<locals>.<lambda> at 0x7f21e832e550>'},
 'prior_order_y': False,
 'prior_outputscale': 2.3163584733185836,
 'prior_type': 'gp'}

In [6]:
bnn_model_checkpoint_dir = "results/tabular_model_bnn.ckpt"
bnn_model_config = {'batch_size': 512,
 'bptt': 50,
 'dropout': 0.5,
 'emsize': 512,
 'epochs': 100,
 'eval_positions': [10, 20, 40],
 'lr': 1.6421403128751275e-05,
 'nhead': 4,
 'nhid_factor': 2,
 'nlayers': 5,
 'num_features': 60,
 'prior_activations': "<class 'torch.nn.modules.activation.Tanh'>",
 'prior_dropout_sampler': {'lambda: 0.0': '<function <lambda> at 0x7f613c1364c0>'},
 'prior_emsize_sampler': {'scaled_beta_sampler_f(2.0, 4.0, 150, 2)': '<function <lambda>.<locals>.<lambda> at 0x7f613c136310>'},
 'prior_is_causal': False,
 'prior_nlayers_sampler': {'lambda: 3': '<function <lambda> at 0x7f613c136790>'},
 'prior_noise_std_gamma_k': 1.8663049257557085,
 'prior_noise_std_gamma_theta': 0.05275478076173361,
 'prior_normalize_by_used_features': False,
 'prior_num_features_used_sampler': {'scaled_beta_sampler_f(1.0, 1.6, max_features, 2)': '<function <lambda>.<locals>.<lambda> at 0x7f613c136550>'},
 'prior_order_y': True,
 'prior_sigma_gamma_k': 3.6187797729244253,
 'prior_sigma_gamma_theta': 0.06773738681062867,
 'prior_type': 'mlp'}

## Loading PFN

In [12]:
model_type = 'bnn'
if model_type == 'gp':
    raise Exception("Not Implemented")
    config = gp_model_config
    checkpoint_dir = gp_model_checkpoint_dir
elif model_type == 'bnn':
    config = bnn_model_config
    checkpoint_dir = bnn_model_checkpoint_dir

model = get_model(config, device, eval_positions, should_train=False)
model_state, _ = torch.load(checkpoint_dir)
model[2].load_state_dict(model_state)
model = model[2]

Using cuda device
DataLoader.__dict__ {'num_steps': 100, 'fuse_x_y': False, 'get_batch_kwargs': {'batch_size': 512, 'seq_len': 50, 'num_features': 60, 'hyperparameters': ('<function <lambda> at 0x7f613c136790>', '<function <lambda>.<locals>.<lambda> at 0x7f613c136310>', "<class 'torch.nn.modules.activation.Tanh'>", <function <lambda>.<locals>.<lambda> at 0x7f56719e5940>, <function <lambda>.<locals>.<lambda> at 0x7f56719e5b80>, '<function <lambda> at 0x7f613c1364c0>', True, '<function <lambda>.<locals>.<lambda> at 0x7f613c136550>', None, False, None, None, None, True, False, None, 0.0), 'batch_size_per_gp_sample': 8}, 'num_features': 60, 'num_outputs': 1}


## Evaluation of PFN and Baselines on all datasets

### Transformer

In [26]:
device = 'cuda'
result = evaluate(ds, model.to(device), 'transformer'
                  , max_features = max_features
                  , bptt=seq_len
                  , eval_position_range=eval_positions
                  , device=device
                  , max_samples=20
                  , rescale_features=config["prior_normalize_by_used_features"]
                  , extend_features=True, plot=False, overwrite=True, save=False)
result

Evaluating wine
	 Eval position 30 done..
Evaluating covertype
	 Eval position 30 done..


{'metric': 'auc',
 'wine_mean_metric_at_30': 0.9587367346938775,
 'wine_time': 0.45734596252441406,
 'covertype_mean_metric_at_30': 0.9624857142857144,
 'covertype_time': 0.4606165885925293,
 'mean_metric_at_30': 0.960611224489796,
 'mean_metric': 0.960611224489796}

### Bayesian NN

In [ ]:
result = evaluate(ds, bayes_net_metric, 'bayes_net'
                  , bptt=seq_len
                  , evaal_position_range=eval_positions
                  , device=device
                  , max_samples=20
                  , overwrite=True
                  , save=False)
result

### Gaussian Process

In [29]:
result = evaluate(ds, gp_metric, 'gp'
                  , bptt=seq_len
                  , evaal_position_range=eval_positions
                  , device=device
                  , max_samples=20
                  , overwrite=True
                  , save=False)
result

Evaluating wine


 20%|██        | 1/5 [00:05<00:22,  5.68s/it]

0.9857142857142858


 40%|████      | 2/5 [00:10<00:14,  4.88s/it]

0.9714285714285714


...

0.7142857142857143


 80%|████████  | 4/5 [00:22<00:05,  5.44s/it]

0.8142857142857143


100%|██████████| 5/5 [00:27<00:00,  5.50s/it]

0.8714285714285714
	 Eval position 30 done..


In [ ]:
result

### Tabnet

In [1403]:
result = evaluate(ds, tabnet_metric, 'tabnet'
                  , bptt=seq_len
                  , eval_position_range=eval_positions
                  , device=device
                  , max_samples=20
                  , overwrite=True
                  , save=False)
result

Evaluating blood-transfusion-service-center


  0%|          | 0/2 [00:00<?, ?it/s]

Device used : cuda
No early stopping will be performed, last training weights will be used.
epoch 0  | loss: 0.89694 |  0:00:00s
epoch 1  | loss: 0.77102 |  0:00:00s
epoch 2  | loss: 0.85065 |  0:00:00s
epoch 3  | loss: 0.55303 |  0:00:00s
epoch 4  | loss: 0.54864 |  0:00:00s
epoch 5  | loss: 0.57434 |  0:00:00s
epoch 6  | loss: 0.53758 |  0:00:00s
epoch 7  | loss: 0.50961 |  0:00:00s
epoch 8  | loss: 0.49373 |  0:00:00s
epoch 9  | loss: 0.48734 |  0:00:00s
epoch 10 | loss: 0.46865 |  0:00:00s
epoch 11 | loss: 0.45546 |  0:00:00s
epoch 12 | loss: 0.44154 |  0:00:00s
epoch 13 | loss: 0.42482 |  0:00:00s


  0%|          | 0/2 [00:01<?, ?it/s]


KeyboardInterrupt: 

### KNN

In [23]:
result = evaluate(ds, knn_metric, 'knn'
                  , bptt=seq_len
                  , eval_position_range=eval_positions
                  , device=device
                  , max_samples=20
                  , overwrite=True
                  , save=False)
result

Evaluating wine


100%|██████████| 20/20 [00:01<00:00, 11.50it/s]


	 Eval position 30 done..
Evaluating covertype


100%|██████████| 20/20 [00:01<00:00, 16.24it/s]

	 Eval position 30 done..


{'metric': 'auc',
 'wine_mean_metric_at_30': 0.8925102040816327,
 'wine_time': 1.7465898990631104,
 'covertype_mean_metric_at_30': 0.792734693877551,
 'covertype_time': 1.247727394104004,
 'mean_metric_at_30': 0.8426224489795919,
 'mean_metric': 0.8426224489795919}

### Logistic Regression

In [25]:
result = evaluate(ds, logistic_metric, 'logistic'
                  , bptt=seq_len
                  , eval_position_range=eval_positions
                  , device=device
                  , max_samples=20
                  , overwrite=True
                  , save=False)
result

Evaluating wine


  5%|▌         | 1/20 [00:01<00:34,  1.83s/it]

{'C': 1e-05, 'fit_intercept': True, 'max_iter': 500, 'penalty': 'none', 'solver': 'saga', 'tol': 0.01}
[4.12326081e-04 9.19611425e-01 3.97752626e-02 9.74932808e-01
 2.92177993e-03 9.98121211e-01 4.40905019e-03 9.33577461e-01
 1.50555390e-01 9.94204349e-01 3.87256524e-04 9.75964168e-01
 6.52074221e-03 5.44187545e-01 2.76466189e-04 7.75604223e-01
 6.95195131e-04 8.77684487e-01 3.72080751e-04 9.98190223e-01
 1.30797581e-02 9.98647814e-01 2.64468869e-01 9.99135067e-01
 2.63580359e-05 6.23341075e-01 1.93786767e-03 9.08927121e-01
 3.61541354e-03 9.98969131e-01 1.74570882e-02 9.08004442e-01
 1.55259675e-02 9.57181495e-01 2.78544674e-03 9.99731021e-01
 7.09553284e-02 2.29517949e-01 3.19418012e-02 6.85265538e-01
 5.43416196e-03 9.67286609e-01 3.30096232e-02 8.83952036e-01
 9.05628129e-04 9.25079429e-01 1.30549388e-03 9.87857426e-01
 1.93246531e-02 7.14752885e-01 6.55393417e-03 9.91341790e-01
 2.71002767e-01 9.72008984e-01 1.95572800e-02 9.70650138e-01
 4.02880801e-04 9.36476310e-01 1.71522394e-

 10%|█         | 2/20 [00:03<00:35,  1.99s/it]

{'C': 1e-05, 'fit_intercept': False, 'max_iter': 500, 'penalty': 'none', 'solver': 'saga', 'tol': 0.01}
[2.34966376e-03 9.97811502e-01 4.42977805e-03 9.99565614e-01
 2.59753692e-03 9.61724168e-01 2.61071324e-03 9.99933616e-01
 1.46586638e-03 9.85141705e-01 1.99387649e-04 9.14871930e-01
 2.31569875e-02 9.77123157e-01 4.55479338e-04 9.98882992e-01
 2.01825925e-03 9.12376267e-01 2.19126373e-01 9.95130404e-01
 5.17808246e-04 9.41544348e-01 5.26481881e-03 5.45981587e-01
 1.29234306e-04 7.59193071e-01 5.28180762e-04 8.64066925e-01
 5.49872741e-04 9.99114519e-01 3.82477700e-03 9.97710659e-01
 3.32963599e-01 9.98419850e-01 2.61277431e-05 5.27080955e-01
 6.53393899e-03 7.76305159e-01 6.04063168e-03 9.98567996e-01
 3.01862055e-02 8.02008437e-01 1.81304029e-02 9.54083311e-01
 7.13274150e-03 9.99326682e-01 1.00578841e-01 2.94002156e-01
 2.58978601e-02 7.49132883e-01 4.50584482e-03 9.77984332e-01
 8.14691986e-02 8.71252875e-01 9.75149695e-04 9.42233545e-01
 9.85537736e-04 9.96837487e-01 1.44663880e

...

 45%|████▌     | 9/20 [00:16<00:20,  1.86s/it]

{'C': 1e-05, 'fit_intercept': True, 'max_iter': 500, 'penalty': 'l2', 'solver': 'saga', 'tol': 1e-10}
[0.4998356  0.50015137 0.49987409 0.50029966 0.49983398 0.50031188
 0.50009655 0.50023837 0.49986525 0.5002024  0.49988854 0.50012421
 0.49990913 0.50033222 0.50001204 0.50011976 0.50011477 0.50028586
 0.49994748 0.50036626 0.50007451 0.50013711 0.50013507 0.50012796
 0.49997282 0.50019476 0.49997082 0.50019112 0.5001566  0.50017832
 0.49986975 0.50027281 0.49993339 0.50016413 0.50001914 0.50030764
 0.50022348 0.50024722 0.50009243 0.500288   0.49992337 0.50020947
 0.50000242 0.50023711 0.49991418 0.50019936 0.50010521 0.50013699
 0.50003493 0.50039762 0.49996634 0.49993259 0.50006249 0.50027367
 0.49986551 0.49999889 0.49985123 0.4999571  0.50000606 0.49985354
 0.49994856 0.49987396 0.4999105  0.49993825 0.49989787 0.49983024
 0.49994391 0.50008364 0.50003337 0.50000305]


 45%|████▌     | 9/20 [00:17<00:21,  1.92s/it]
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/anon/miniconda3/envs/prior-fitting/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 3441, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_62303/3352290182.py", line 1, in <module>
    result = evaluate(ds, logistic_acc, 'logistic'
  File "/home/anon/prior-fitting/tabular.py", line 195, in evaluate
    ds_result = evaluate_dataset(X.to(device), y.to(device), categorical_feats, model, bptt, eval_position_range,
  File "/home/anon/prior-fitting/tabular.py", line 227, in evaluate_dataset
    r = evaluate_position(X, y, categorical_feats, model, bptt, eval_position, rescale_features=rescale_features,
  File "/home/anon/prior-fitting/tabular.py", line 314, in evaluate_position
    acc_eval_pos, outputs = batch_pred(model, eval_xs, eval_ys, categorical_feats, start=eval_position)
  File "/home/anon/prior-fitting/tabular.py", line 329, in batch_pred
    acc, output = acc_function(eval_x

TypeError: object of type 'NoneType' has no len()

### Ridge Classification

In [1303]:
result = evaluate(ds, ridge_metric, 'ridge'
                  , bptt=seq_len
                  , eval_position_range=eval_positions
                  , device=device
                  , max_samples=20
                  , overwrite=True
                  , save=False)
result

Evaluating kr-vs-kp


  5%|▌         | 1/20 [00:00<00:02,  6.91it/s]

0.8428571428571429


 10%|█         | 2/20 [00:00<00:02,  6.81it/s]

1.0


...

1.0


 75%|███████▌  | 15/20 [00:02<00:00,  6.64it/s]

1.0


 75%|███████▌  | 15/20 [00:02<00:00,  6.37it/s]


KeyboardInterrupt: 

### XG Boost

In [1346]:
result = evaluate(ds, ridge_metric, 'ridge'
                  , bptt=seq_len
                  , eval_position_range=eval_positions
                  , device=device
                  , max_samples=20
                  , overwrite=True
                  , save=False)
result

Evaluating kr-vs-kp


  0%|          | 0/2 [00:00<?, ?it/s]

Fitting 2 folds for each of 1 candidates, totalling 2 fits
[17:30:32] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


 50%|█████     | 1/2 [00:04<00:04,  4.52s/it]

{}
1.0
Fitting 2 folds for each of 1 candidates, totalling 2 fits
[17:30:36] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


...

  0%|          | 0/2 [00:00<?, ?it/s]

Fitting 2 folds for each of 1 candidates, totalling 2 fits


  0%|          | 0/2 [00:01<?, ?it/s]
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



[17:30:50] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[17:30:35] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[CV] END .................................................... total time=   1.1s
[17:30:39] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[CV] END .................................................... total time=   1.2s
[17:30:46] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default

TypeError: object of type 'NoneType' has no len()

In [ ]:
result

### Catboost

In [1338]:
result = evaluate(ds, catboost_metric, 'catboost'
                  , bptt=seq_len
                  , eval_position_range=eval_positions
                  , device=device
                  , max_samples=20
                  , overwrite=True
                  , save=False)
result

Evaluating kr-vs-kp


100%|██████████| 1/1 [00:00<00:00,  3.91it/s]


0.6428571428571429
	 Eval position 30 done..
Evaluating credit-g


100%|██████████| 1/1 [00:00<00:00,  5.58it/s]

0.5571428571428572
	 Eval position 30 done..
Evaluating vehicle


...

0.9142857142857143
	 Eval position 30 done..
Evaluating jannis


100%|██████████| 1/1 [00:00<00:00,  4.91it/s]


0.5571428571428572
	 Eval position 30 done..
Evaluating helena


  0%|          | 0/1 [00:00<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
result

## Running bayesian inference on one dataset

In [47]:
# NOTE: Dataset must be ordered 1,0,1,0,...
one_ds = ds[0]
one_ds

['wine',
 tensor([[1.1640e+01, 2.0600e+00, 2.4600e+00,  ..., 1.0000e+00, 2.7500e+00,
          6.8000e+02],
         [1.3860e+01, 1.3500e+00, 2.2700e+00,  ..., 1.0100e+00, 3.5500e+00,
          1.0450e+03],
         [1.2340e+01, 2.4500e+00, 2.4600e+00,  ..., 8.0000e-01, 3.3800e+00,
          4.3800e+02],
         ...,
         [1.3450e+01, 3.7000e+00, 2.6000e+00,  ..., 8.5000e-01, 1.5600e+00,
          6.9500e+02],
         [1.1560e+01, 2.0500e+00, 3.2300e+00,  ..., 9.3000e-01, 3.6900e+00,
          4.6500e+02],
         [1.2820e+01, 3.3700e+00, 2.3000e+00,  ..., 7.2000e-01, 1.7500e+00,
          6.8500e+02]]),
 tensor([0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1.,
         0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1.,
         0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1.,
         0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1.,
         0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 

In [53]:
result_list = []
eval_position = eval_positions[0]
rescale_features = max_features / one_ds[1].shape[1] if config['prior_normalize_by_used_features'] else 1

model = model.to(device)
model = model.eval()

# Data to run inference on
eval_xs = one_ds[1][0:50]
# Extending dataset to standardized feature length
eval_xs = torch.cat([eval_xs, torch.zeros((eval_xs.shape[0], max_features - eval_xs.shape[1]))], -1).to(device)
eval_ys = one_ds[2][0:50].to(device)

for i, pos in enumerate(range(eval_position, eval_xs.shape[0])):
    eval_x = torch.cat([eval_xs[:eval_position], eval_xs[pos].unsqueeze(0)])
    eval_y = eval_ys[:eval_position]

    # Center data using training positions so that it matches priors
    mean = eval_x.mean(0)
    std = eval_x.std(0) + .000001
    eval_x = (eval_x - mean) / std
    eval_x = eval_x / rescale_features

    result_list += [torch.sigmoid(model((eval_x.unsqueeze(1), eval_y.unsqueeze(1).float()), single_eval_pos=eval_position)).squeeze(-1).detach().cpu().numpy()[0]]

print(np.array(result_list))

[[0.07891185]
 [0.9558054 ]
 [0.05336431]
 [0.98093307]
 [0.07384931]
 [0.8429412 ]
 [0.03347415]
 [0.973169  ]
 [0.04434767]
 [0.9206543 ]
 [0.03534983]
 [0.7528224 ]
 [0.23227222]
 [0.96119374]
 [0.20475379]
 [0.97402555]
 [0.04753578]
 [0.8418446 ]
 [0.20400934]
 [0.9496556 ]]


In [1229]:
result = evaluate(ds, bayes_net_acc, 'bayes_net'
                  , bptt=seq_len
                  , eval_position_range=eval_positions
                  , device=device
                  , max_samples=20
                  , overwrite=True
                  , save=False)
result

Evaluating kr-vs-kp


  0%|          | 0/5 [00:00<?, ?it/s]

Fitting 2 folds for each of 8 candidates, totalling 16 fits
5



100%|██████████| 300/300 [00:04<00:00, 72.61it/s] 


[CV 1/2] END .................embed=5, lr=0.001;, score=1.000 total time=   4.8s
5


...

[CV 2/2] END ................embed=5, lr=0.0001;, score=1.000 total time=   2.0s
10



100%|██████████| 300/300 [00:01<00:00, 160.18it/s]


[CV 1/2] END ................embed=10, lr=0.001;, score=1.000 total time=   2.2s
10



  0%|          | 0/5 [00:14<?, ?it/s]


KeyboardInterrupt: 